Trong nhiều ứng dụng, dữ liệu có thể nằm rải rác trên một số tệp hoặc cơ sở dữ liệu hoặc được sắp xếp ở dạng không dễ phân tích. Chương này tập trung vào các công cụ giúp kết hợp, nối và sắp xếp lại dữ liệu.

Đầu tiên, tôi giới thiệu khái niệm`hierarchical indexing in pandas `, được sử dụng rộng rãi trong một số hoạt động này. Sau đó, tôi đi sâu vào các thao tác dữ liệu cụ thể. Bạn có thể thấy các cách sử dụng khác nhau của các công cụ này trong ”

## 8.1 Hierarchical Indexing


**Hierarchical Indexing** là một tính năng quan trọng của pandas cho phép bạn có nhiều (hai hoặc nhiều) cấp chỉ mục trên một trục. Hơi trừu tượng, nó cung cấp một cách để bạn làm việc với dữ liệu có chiều cao hơn ở dạng chiều thấp hơn. Hãy bắt đầu với một ví dụ đơn giản; tạo Chuỗi với danh sách các danh sách (hoặc mảng) làm chỉ mục:

In [1]:
import numpy as np
import pandas as pd

In [2]:

data = pd.Series(np.random.randn(9),
                index=[
                    ['a','a','a','b','b','c','c','d','d'],
                    [1,2,3,1,3,1,2,2,3]
                ])
data

a  1    0.039320
   2   -0.885046
   3    0.150814
b  1    0.895163
   3    0.657531
c  1    0.598231
   2   -1.017074
d  2   -1.291932
   3   -2.092272
dtype: float64

Những gì bạn thấy là 1 views rất đẹp của Series với nhiều loại Index,


In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

Với một đối tượng dùng `hierarchically indexed` object, có thể gọi là chỉ mục từng phần, cho phép bạn chọn chính xác tập hợp con của dữ liệu

In [4]:
data

a  1    0.039320
   2   -0.885046
   3    0.150814
b  1    0.895163
   3    0.657531
c  1    0.598231
   2   -1.017074
d  2   -1.291932
   3   -2.092272
dtype: float64

In [5]:
data['b']

1    0.895163
3    0.657531
dtype: float64

In [6]:
data['b':'c']

b  1    0.895163
   3    0.657531
c  1    0.598231
   2   -1.017074
dtype: float64

In [7]:
data.loc[['b','d']]

b  1    0.895163
   3    0.657531
d  2   -1.291932
   3   -2.092272
dtype: float64

Selections thậm chí có thể đào sâu được level


In [8]:
data.loc[:,2]

a   -0.885046
c   -1.017074
d   -1.291932
dtype: float64

`Hierarchical Indexing` đóng một vai trò quan trọng trong việc định hình lại dữ liệu và các hoạt động dựa trên nhóm như hình thành một bảng tổng hợp. Ví dụ: bạn có thể sắp xếp lại dữ liệu thành DataFrame bằng cách sử dụng phương pháp bỏ đóng gói của nó

In [9]:
data.unstack()

,1,2,3
a,0.039320,-0.885046,0.150814
b,0.895163,NaN,0.657531
c,0.598231,-1.017074,NaN
d,NaN,-1.291932,-2.092272


Để ngược lại nó nữa thì dùng `stack()`

In [10]:
data.unstack().stack()

a  1    0.039320
   2   -0.885046
   3    0.150814
b  1    0.895163
   3    0.657531
c  1    0.598231
   2   -1.017074
d  2   -1.291932
   3   -2.092272
dtype: float64

chúng ta có thể cho `hierarchical index` dùng cho cột

In [11]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index=[
                        ['a','a','b','b'],
                        [1,2,1,2]
                    ],
                     columns=[
                         ['Ohio','Ohio','Colorado'],
                         ['Green','Red','Green']
                     ])

In [13]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [14]:
frame.index.names = ['key1','key2']
frame.columns.names = ['state','color']

In [15]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

### Reordering and Sorting Levels

Đôi khi bạn sẽ cần phải sắp xếp lại thứ tự của các cấp trên một trục hoặc sắp xếp dữ liệu theo các giá trị trong một cấp cụ thể. `Swaplevel` nhận hai số cấp hoặc tên và trả về một đối tượng mới với các cấp được hoán đổi cho nhau (nhưng dữ liệu thì không thay đổi): ”

In [16]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [17]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

Mặt khác, `sort_index` sắp xếp dữ liệu chỉ sử dụng các giá trị trong một cấp duy nhất. Khi hoán đổi cấp độ, không có gì lạ khi cũng sử dụng sort_index để kết quả được sắp xếp từ vựng theo cấp đã chỉ định

In [18]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [19]:
frame.swaplevel(0,1).sort_index(level=0)


state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

***Hiệu suất lựa chọn dữ liệu tốt hơn nhiều trên các đối tượng được `hierarchically indexed` nếu chỉ mục được sắp xếp theo từ điển bắt đầu bằng cấp ngoài cùng - nghĩa là kết quả của việc gọi sort_index (level = 0) hoặc `sort_index` ().***

## Summary Statistics by Level

In [21]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [20]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [22]:
frame.sum(level='color',axis = 1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Index with a DataFrame's columns

“Không có gì lạ khi bạn muốn sử dụng một hoặc nhiều cột từ DataFrame làm chỉ mục hàng; cách khác, bạn có thể muốn di chuyển chỉ mục hàng vào các cột của DataFrame. Đây là một ví dụ về DataFrame ”

In [24]:
frame = pd.DataFrame(
{
    'a':range(7),'b':range(7,0,-1),
    'c': ['one','one','one','two','two','two','two'],
    'd':[0,1,2,0,1,2,3]
}
)

In [25]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame's `set_index` function sẽ tạo 1 DataFrame sử dung 1 hoặc nhiều columns là index

In [26]:
frame2 = frame.set_index(['c','d'])

In [27]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

 See set_index add arguments `drop=False` 

In [29]:
frame.set_index(['c','d'],drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

`reset_index`

In [30]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 Combining adn Merging Datasets 

* `pandas.merge` connects rows in DataFrames based on one or more keys,

* `pandas.concat` chia ra hoawj cho va

* `combine_frist` Instance method cho phép nối các dữ liệu chồng chéo với nhau để điền vào các giá trị còn thiếu trong một đối tượng với các giá trị từ một đối tượng khác."

### Database-Style DataFrame Joins 

In [31]:
df1 = pd.DataFrame({
    'key':['b','b','a','c','a','a','b'],
    'data1': range(7)
})

In [32]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [36]:
df2 = pd.DataFrame({
    'key': ['a','b','d'],
    'data2': range(3)
})

In [37]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [34]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [35]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


using key chỉ đinh cái gì mà nó merge

In [39]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Nếu những tên khác và mỗi object, và bạn có thể chỉnh định  nó ngay lập tức

In [40]:
df3 = pd.DataFrame({
    'lkey': ['b','b','a','c','a','a','b'],
    'data1': range(7)
})
df4 = pd.DataFrame({
    'rkey':['a','b','d'],
    'data2': range(3)
})


In [41]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [42]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [43]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


default `megre` is **inner-join in SQL**

* `inner` : Sử dụng key kết hợp từ cả 2 bảng
* `left`: Phép nối left-join
* `right`: Phép nối right-join
* `out-put`: full join :D 

**Many-to-Many**  Example: 

In [44]:
df1 = pd.DataFrame(
{'key': ['b','b','a','c','a','b'],
'data1': range(6)}
)
df2 = pd.DataFrame({
    'key': ['a','b','a','b','d'],
    'data2': range(5)
})

In [45]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [46]:

df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [47]:
pd.merge(df1,df2,on='key',how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [49]:
pd.merge(df1,df2,how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


**merge với nhiều keys**

In [56]:
left= pd.DataFrame({
    'key1': ['foo','foo','bar'],
    'key2': ['one','two','one'],
    'lval': [1,2,3]
})

In [52]:
right = pd.DataFrame({
    'key1': ['foo','foo','bar','bar'],
    'key2': ['one','one','one','two'],
'rval': [4,5,6,7]
})

In [53]:
left

,key1,key,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [54]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [57]:
pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


## Merging on Index

Trong một số trường hợp, (các) khóa hợp nhất trong DataFrame sẽ được tìm thấy trong chỉ mục của nó. Trong trường hợp này, bạn có thể chuyển `left_index = True` hoặc `right_index = True` (hoặc cả hai) để chỉ ra rằng chỉ mục nên được sử dụng làm khóa hợp nhất: ”

In [59]:
left1 = pd.DataFrame({'key': ['a','b','a','a','b','c'],
                     'value': range(6)})

In [61]:
right1 = pd.DataFrame({
    'group_val': [3.5,7],
},index=['a','b'])

In [66]:
left1


,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [67]:
right1

,group_val
a,3.5
b,7.0


In [72]:
pd.merge(left1,right1,left_on='key',right_index=True,how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


Với `hierarchically indexed` data, nghĩ rất phức tạp và để **join** và lý do là rất nhiều key

In [75]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                                   'Nevada', 'Nevada'],
                          'key2': [2000, 2001, 2002, 2001, 2002],
                          'data': np.arange(5.)})
 

In [76]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                          index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                                 'Ohio', 'Ohio'],
                                 [2001, 2000, 2000, 2000, 2001, 2002]],
                          columns=['event1', 'event2'])

In [77]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [78]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [79]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


### Concatenating Along an Axis

Một loại hoạt động kết hợp dữ liệu khác được gọi thay thế cho nhau là `concatenation, binding, ỏ stacking `. Hàm nối của NumPy có thể thực hiện điều này với mảng NumPy

In [81]:
arr = np.arange(12).reshape((3,4))

In [82]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [83]:
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [84]:
np.concatenate([arr,arr])

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [85]:
s1 = pd.Series([0,1],index=['a','b'])
s2 = pd.Series([2,3,4],index=['c','d','e'])
s3 = pd.Series([5,6],index=['f','g'])

In [86]:
s1

a    0
b    1
dtype: int64

In [87]:
s2

c    2
d    3
e    4
dtype: int64

In [88]:
s3

f    5
g    6
dtype: int64

In [89]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [90]:
pd.concat([s1,s2,s3],axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


Trong trường hợp này, không có sự chồng chéo trên trục khác, như bạn có thể thấy là liên kết được sắp xếp (liên kết `outer join`) của các chỉ mục. Thay vào đó, bạn có thể cắt chúng bằng cách truyền `join = 'inner'`

In [91]:
s4 = pd.concat([s1,s3])

In [92]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [93]:
pd.concat([s1,s4],axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [94]:
result = pd.concat([s1,s1,s3],keys=['one','two','three'])

In [95]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [96]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [97]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                columns=['one', 'two'])




In [98]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [99]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                    columns=['three', 'four'])

In [100]:
df2

,three,four
a,5,6
c,7,8


In [101]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [102]:
df2

,three,four
a,5,6
c,7,8


In [104]:
pd.concat([df1,df2],axis=1,keys=['level1','level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [105]:
## Passing like dictionary
pd.concat({
    'level1': df1,
    'level2': df2
},axis=1)





level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

### Combining Data with Overlap

Có một tình huống kết hợp dữ liệu khác không thể được biểu thị dưới dạng hoạt động hợp nhất hoặc ghép nối. Bạn có thể có hai tập dữ liệu có các chỉ mục trùng lặp toàn bộ hoặc một phần. Như một ví dụ thúc đẩy, hãy xem xét `hàm NumPy’s where`, thực hiện tương đương với hướng mảng của một biểu thức `if-else`:

In [106]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
                  index=['f', 'e', 'd', 'c', 'b', 'a'])

In [107]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [108]:
b = pd.Series(np.arange(len(a), dtype=np.float64),
                 index=['f', 'e', 'd', 'c', 'b', 'a'])

In [109]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [110]:
b[-1] = np.nan

In [111]:
np.where(pd.isnull(a),b,a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

Serise có 1 hàm `combine_first`, xử lí tương đương với hàm `where`

In [112]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

Với DataFrames , `combine_first`  làm những điều tương tự từ cột sang cột , có thể nghĩ là`patching`

In [113]:
df1 = pd.DataFrame({
    'a': [1.,np.nan,5.,np.nan],
    'b':[np.nan,2.,np.nan,6.],
    'c': range(2,18,4)
})

In [114]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [115]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                     'b': [np.nan, 3., 4., 6., 8.]})


In [116]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [117]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


### 8.3 Reshaping and Pivoting 
“Có một số thao tác cơ bản để sắp xếp lại dữ liệu dạng bảng. Chúng được gọi  là  `reshape` hoặc`pivot`. "

### Reshape with Hierarchical Indexing

Lập chỉ mục phân cấp cung cấp một cách nhất quán để sắp xếp lại dữ liệu trong DataFrame. Có hai hành động chính:

**stack** xoay từ cột sang hàng 

**unstack** xoay các hàng thành cột